In [1]:
import pandas as pd
import numpy as np
import json
import os
from tabulate import tabulate

In [5]:
cutoff = 16
potential = 'AHO'
folder_path = r"C:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\SUSY\SUSY QM\PennyLane\VQD\Destructive SWAP\VaryBeta\{}".format(potential)
betas = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]

In [6]:
converged_only = True
all_data = []

for beta in betas:

    median = []
    mean = []
    min = []
    e0 = []
    e1 = []
    e2 = []
    ratios = []

    data_path = folder_path + "\\{}\\{}_{}.json".format(beta,potential, cutoff)
    
    with open(data_path, 'r') as file:
        data = json.load(file)

    num_VQD = data['num_VQD']

    converged = [all(x) for x in data['success']]
    converged_indices = [i for i, success in enumerate(converged) if success]
    converged_runs = sum(converged)

    if converged_only:
        results = [data['results'][i] for i in converged_indices]
    else:
        results = data['results']

    success_i = [i for i, success in enumerate(data['success']) if all(success)]
    successful_results = [data['results'][i] for i in success_i]
    energies =np.sort(successful_results)

    run_times = data['run_times']#[data['run_times'][i] for i in converged_indices]
    time_series = np.array([pd.Timedelta(t).total_seconds() for t in run_times])
    mean_vqe_time = np.mean(time_series)
    time_taken = np.mean(mean_vqe_time)

    for e in energies:
        
        e0.append(e[0])
        e1.append(e[1])
        e2.append(e[2])
        r = abs((e[2] - e[1]) / (e[2] - e[0]))
        ratios.append(r)

    e0_exact = data['exact_eigenvalues'][0]
    e1_exact = data['exact_eigenvalues'][1]
    e2_exact = data['exact_eigenvalues'][2]
    exact_ratio = abs((e2_exact - e1_exact) / (e2_exact - e0_exact))

    row = {
    "beta": beta,
    'Converged Runs': f"{converged_runs}/100",
    'Mean VQD time (s)': f"{time_taken:.2f}",
    #"e0_exact": e0_exact,
    #"e1_exact": e1_exact,
    #"e2_exact": e2_exact,
    #"e0": np.median(e0),
    #"e1": np.median(e1),
    #"e2": np.median(e2),
    "E01": e1_exact - e0_exact,
    "E12": e2_exact - e1_exact,
    "exact_ratio": exact_ratio,
    "median_ratio": np.median(ratios),
    "delta_ratio": abs(exact_ratio-np.median(ratios))
    #"mean_ratio": np.mean(ratios),
    #"min_ratio": np.min(ratios),
    }

    all_data.append(row)

In [7]:
df = pd.DataFrame(all_data)
df

,beta,Converged Runs,Mean VQD time (s),E01,E12,exact_ratio,median_ratio,delta_ratio
0,0.5,100/100,378.63,1.678661,0.008887,0.005266,0.500174,0.494908
1,1.0,99/100,356.30,1.678661,0.008887,0.005266,0.406623,0.401357
2,1.5,100/100,321.49,1.678661,0.008887,0.005266,0.113939,0.108673
3,2.0,96/100,404.03,1.678661,0.008887,0.005266,0.010586,0.005320
4,2.5,100/100,342.52,1.678661,0.008887,0.005266,0.009760,0.004494
5,3.0,99/100,399.05,1.678661,0.008887,0.005266,0.011603,0.006337


In [8]:
latex_table = tabulate(df.values, headers=df.columns, tablefmt="latex", floatfmt=".3e")
print(latex_table)

\begin{tabular}{rlrrrrrr}
\hline
      beta & Converged Runs   &   Mean VQD time (s) &       E01 &       E12 &   exact\_ratio &   median\_ratio &   delta\_ratio \\
\hline
 5.000e-01 & 100/100          &           3.786e+02 & 1.679e+00 & 8.887e-03 &     5.266e-03 &      5.002e-01 &     4.949e-01 \\
 1.000e+00 & 99/100           &           3.563e+02 & 1.679e+00 & 8.887e-03 &     5.266e-03 &      4.066e-01 &     4.014e-01 \\
 1.500e+00 & 100/100          &           3.215e+02 & 1.679e+00 & 8.887e-03 &     5.266e-03 &      1.139e-01 &     1.087e-01 \\
 2.000e+00 & 96/100           &           4.040e+02 & 1.679e+00 & 8.887e-03 &     5.266e-03 &      1.059e-02 &     5.320e-03 \\
 2.500e+00 & 100/100          &           3.425e+02 & 1.679e+00 & 8.887e-03 &     5.266e-03 &      9.760e-03 &     4.494e-03 \\
 3.000e+00 & 99/100           &           3.991e+02 & 1.679e+00 & 8.887e-03 &     5.266e-03 &      1.160e-02 &     6.337e-03 \\
\hline
\end{tabular}
